## Fact Orders

In [0]:
df = spark.sql("select * from databricks_cata.silver.orders")
display(df)

In [0]:
df_dimcus = spark.sql("select DimCustomerkey, customer_id as dim_customer_id from databricks_cata.gold.dimcustomer")

df_dimpro = spark.sql("select product_id as Dimproductkey, product_id as dim_product_id from databricks_cata.gold.dimproducts")




### fact dataframe

In [0]:
df_fact = df.join(df_dimcus, df['customer_id'] == df_dimcus['dim_customer_id'], how="left").join(df_dimpro, df['product_id'] == df_dimpro['dim_product_id'], how="left")

df_fact_new = df_fact.drop("dim_customer_id").drop("dim_product_id", "customer_id", "product_id")


In [0]:
df_fact_new.display()

upsert on the fact table

In [0]:
from  delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_cata.gold.Factorders"):
    dlt_obj = DeltaTable.forName(spark, "databricks_cata.gold.Factorders")

    dlt_obj.alias("target").merge(df_fact_new.alias("source"), "target.order_id = source.order_id and target.DimCustomerkey = source.DimCustomerkey and target.Dimproductkey = source.Dimproductkey")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

else:

    df_fact_new.write.format("delta").option("path", "abfss://gold@rtgpadlsgen2.dfs.core.windows.net/Factorders").saveAsTable("databricks_cata.gold.Factorders")

In [0]:
%sql
select * from databricks_cata.gold.Factorders